# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Output File (CSV)
output_data_file = "../output_data/city_weather.csv"

# pull prevously generated and saved output data from csv
weather_df = pd.read_csv(output_data_file)

weather_df


,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed
0,0,Vaini,TO,1610067889,-21.2000,-175.2000,86.00,86.00,79,40,10.36
1,1,Ribeira Grande,PT,1610067890,38.5167,-28.7000,57.42,57.42,66,7,20.74
2,2,Busselton,AU,1610067891,-33.6500,115.3333,88.00,88.00,30,0,5.99
3,3,Butaritari,KI,1610067893,3.0707,172.7902,81.50,81.50,81,100,19.98
4,4,Mitsamiouli,KM,1610067894,-11.3847,43.2844,81.00,81.00,81,16,12.30
...,...,...,...,...,...,...,...,...,...,...,...
576,576,Morehead,US,1610068640,37.2711,-87.1764,38.03,39.00,56,75,4.61
577,577,Leh,IN,1610068642,34.1667,77.5833,8.47,8.47,78,93,2.84
578,578,Alugan,PH,1610068643,12.2188,125.4808,78.10,78.10,93,100,11.01
579,579,Uruaçu,BR,1610068644,-14.5247,-49.1408,73.80,73.80,84,60,2.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["humidity"].astype(float)

In [18]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Less than or equal to 15 % cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# get low temp outliers index
temp_outlier_df = weather_df.loc[weather_df["max temperature"] < 70]
weather_df.drop(weather_df["city_id"][temp_outlier_df.index.tolist()], inplace = True)    

temp_outlier_df = weather_df.loc[weather_df["max temperature"] > 80]
weather_df.drop(weather_df["city_id"][temp_outlier_df.index.tolist()], inplace = True)    
  
wind_outlier_df = weather_df.loc[weather_df["wind speed"] > 10]
weather_df.drop(weather_df["city_id"][wind_outlier_df.index.tolist()], inplace = True)    

cloud_outlier_df = weather_df.loc[weather_df["cloudiness"] >= 15]
weather_df.drop(weather_df["city_id"][cloud_outlier_df.index.tolist()], inplace = True)    
      

vacation_weather_df = weather_df
vacation_weather_df

,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed
19,19,Lakes Entrance,AU,1610067915,-37.8811,147.9810,71.91,73.99,64,0,5.99
20,20,Cabo San Lucas,MX,1610067713,22.8909,-109.9124,74.46,75.20,43,1,8.05
27,27,Arraial do Cabo,BR,1610067928,-22.9661,-42.0278,77.00,77.00,91,0,5.75
34,34,Port Lincoln,AU,1610067936,-34.7333,135.8667,71.20,71.20,63,0,9.89
58,58,San Luis,AR,1610067969,-33.2950,-66.3356,72.34,72.34,32,0,3.74
108,108,Takoradze,GH,1610068032,4.8845,-1.7554,79.18,79.18,87,12,8.50
110,110,Kapaa,US,1610068034,22.0752,-159.3190,78.37,78.80,69,1,5.75
119,119,New Norfolk,AU,1610068049,-42.7826,147.0587,75.83,79.00,37,1,3.00
137,137,Canavieiras,BR,1610068070,-15.6750,-38.9472,75.58,75.58,84,0,6.78
146,146,Maragogi,BR,1610068081,-9.0122,-35.2225,77.76,77.76,81,7,8.75


In [20]:
vacation_cities_count = vacation_weather_df["city_id"].nunique()
print(f"Wow! {vacation_cities_count} cities with ideal vacation weather. Let's find a place to stay...")

Wow! 37 cities with ideal vacation weather. Let's find a place to stay...


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
city_id = []
hotel_name = []
hotel_address = []
hotel_rating = []
hotel_lat = []
hotel_lng = []


# find the closest hotel/lodging for each city location
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",    
    "key": g_key        
}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_weather_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotels_response = requests.get(base_url, params=params)
    hotels_json = hotels_response.json()
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    try:
        # print(json.dumps(hotels_json, indent=4, sort_keys=True))
        if ((hotels_json["results"][0]["name"] != "") & (hotels_json["results"][0]["vicinity"] != "") & (hotels_json["results"][0]["rating"] > 0)):
            city_id.append(index)
            hotel_name.append(hotels_json["results"][0]["name"])            
            hotel_address.append(hotels_json["results"][0]["vicinity"])
            hotel_rating.append(hotels_json["results"][0]["rating"])          
            print(f"Hotel Index {index} Added")
          
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    except (KeyError, IndexError):
        print(f"Hotel Index {index} Missing field/result... skipping.")
    

  



Hotel Index 19 Added
Hotel Index 20 Added
Hotel Index 27 Added
Hotel Index 34 Added
Hotel Index 58 Added
Hotel Index 108 Added
Hotel Index 110 Added
Hotel Index 119 Added
Hotel Index 137 Added
Hotel Index 146 Added
Hotel Index 159 Added
Hotel Index 169 Added
Hotel Index 170 Added
Hotel Index 172 Added
Hotel Index 195 Added
Hotel Index 199 Added
Hotel Index 213 Added
Hotel Index 259 Added
Hotel Index 287 Added
Hotel Index 296 Added
Hotel Index 314 Added
Hotel Index 346 Missing field/result... skipping.
Hotel Index 354 Added
Hotel Index 377 Added
Hotel Index 382 Added
Hotel Index 384 Added
Hotel Index 399 Missing field/result... skipping.
Hotel Index 419 Added
Hotel Index 421 Added
Hotel Index 489 Missing field/result... skipping.
Hotel Index 506 Added
Hotel Index 510 Missing field/result... skipping.
Hotel Index 517 Added
Hotel Index 520 Missing field/result... skipping.
Hotel Index 543 Added
Hotel Index 554 Missing field/result... skipping.
Hotel Index 562 Added


In [33]:
# create a data frame for hotel_name, hotel_address, hotel_rating

hotel_dict = {
    "city_id": city_id,
    "Hotel Name": hotel_name,
    "Hotel Address": hotel_address,
    "Hotel Rating": hotel_rating
}

hotel_df = pd.DataFrame(hotel_dict)          
    
hotel_df  


,city_id,Hotel Name,Hotel Address,Hotel Rating
0,19,The Esplanade Resort & Spa,"1 Esplanade, Lakes Entrance",4.1
1,20,Hotel Tesoro Los Cabos,"Boulevard Paseo de la Marina LT 9 y 10, Marina...",4.4
2,27,Pousada Porto Praia,"Rua Vilas Bôas, 161 - Macedônia, Arraial do Cabo",4.2
3,34,Port Lincoln Hotel,"1 Lincoln Highway, Port Lincoln",4.3
4,58,Huarpes Hotel,"Belgrano 1568, GJZ",3.8
5,108,Raybow International Hotel,"906 Beach Road, Residential Area, Sekondi-Tako...",4.2
6,110,Sheraton Kauai Resort at Coconut Beach,"650 Aleka Loop, Kapaʻa",4.1
7,119,The Shingles Riverside Cottages,"117 Glenora Road, New Norfolk",4.7
8,137,Bahiadomizil,"Avenida Beira Mar, 1605 - Praia da Costa, Cana...",4.4
9,146,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124, s/n - Sitio Carió...",4.7


In [23]:
# print(json.dumps(name_address, indent=4, sort_keys=True))

In [34]:
merge_vacations_df = pd.merge(vacation_weather_df, hotel_df, on="city_id")

# Store 'Lat' and 'Lng' into  locations 
locations = merge_vacations_df[["latitude", "longitude"]].astype(float)
merge_vacations_df

,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed,Hotel Name,Hotel Address,Hotel Rating
0,19,Lakes Entrance,AU,1610067915,-37.8811,147.9810,71.91,73.99,64,0,5.99,The Esplanade Resort & Spa,"1 Esplanade, Lakes Entrance",4.1
1,20,Cabo San Lucas,MX,1610067713,22.8909,-109.9124,74.46,75.20,43,1,8.05,Hotel Tesoro Los Cabos,"Boulevard Paseo de la Marina LT 9 y 10, Marina...",4.4
2,27,Arraial do Cabo,BR,1610067928,-22.9661,-42.0278,77.00,77.00,91,0,5.75,Pousada Porto Praia,"Rua Vilas Bôas, 161 - Macedônia, Arraial do Cabo",4.2
3,34,Port Lincoln,AU,1610067936,-34.7333,135.8667,71.20,71.20,63,0,9.89,Port Lincoln Hotel,"1 Lincoln Highway, Port Lincoln",4.3
4,58,San Luis,AR,1610067969,-33.2950,-66.3356,72.34,72.34,32,0,3.74,Huarpes Hotel,"Belgrano 1568, GJZ",3.8
5,108,Takoradze,GH,1610068032,4.8845,-1.7554,79.18,79.18,87,12,8.50,Raybow International Hotel,"906 Beach Road, Residential Area, Sekondi-Tako...",4.2
6,110,Kapaa,US,1610068034,22.0752,-159.3190,78.37,78.80,69,1,5.75,Sheraton Kauai Resort at Coconut Beach,"650 Aleka Loop, Kapaʻa",4.1
7,119,New Norfolk,AU,1610068049,-42.7826,147.0587,75.83,79.00,37,1,3.00,The Shingles Riverside Cottages,"117 Glenora Road, New Norfolk",4.7
8,137,Canavieiras,BR,1610068070,-15.6750,-38.9472,75.58,75.58,84,0,6.78,Bahiadomizil,"Avenida Beira Mar, 1605 - Praia da Costa, Cana...",4.4
9,146,Maragogi,BR,1610068081,-9.0122,-35.2225,77.76,77.76,81,7,8.75,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124, s/n - Sitio Carió...",4.7


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in merge_vacations_df.iterrows()]
locations = merge_vacations_df[["latitude", "longitude"]]


In [36]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5, info_box_content=hotel_info
)

fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))